<h1 align="center">Computational Methods in Environmental Engineering</h1>
<h2 align="center">Lecture #8</h2>
<h3 align="center">9 Mar 2023</h3>



## Broadbcasting



In [1]:
import numpy as np

Form of vectorization that leads to faster code by replacing explicit loops



In [2]:
arr = np.arange(7)
arr * 4 + arr

array([ 0,  5, 10, 15, 20, 25, 30])

What happens when array shapes don't match?



In [3]:
arr = np.random.randn(4, 3)
arr

array([[-0.39045494,  1.85653571,  0.65650551],
       [ 0.41268673,  0.35831446,  0.60322171],
       [-0.30261753,  0.25795829,  0.72414677],
       [ 0.04237356, -0.29257525,  1.34286772]])

Let's subtract the column means



In [4]:
arr.mean(axis=0)

array([-0.05950305,  0.5450583 ,  0.83168543])

In [5]:
arrm = arr - arr.mean(0)
arrm

array([[-0.3309519 ,  1.31147741, -0.17517992],
       [ 0.47218978, -0.18674384, -0.22846371],
       [-0.24311448, -0.28710002, -0.10753866],
       [ 0.1018766 , -0.83763355,  0.51118229]])

In [6]:
arrm.mean(0)

array([-1.04083409e-17, -5.55111512e-17,  5.55111512e-17])

Visualizing Numpy broadcasting

<center><img src="https://i.imgur.com/sodaxEF.png"/></center>



Broadcasting works in higher dimensions as well

<center><img src="https://i.imgur.com/og1iyEQ.png"/></center>

Let's try to subtract the row means this time



In [7]:
arr.mean(1)

array([0.70752876, 0.4580743 , 0.22649584, 0.36422201])

In [8]:
arr - arr.mean(1)

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

  The broadcasting rule
>   Two arrays are compatible for broadcasting if for each trailing dimension (i.e., starting
>   from the end) the axis lengths match or if either of the lengths is 1. Broadcasting is
>   then performed over the missing or length 1 dimensions.



In [9]:
arr - arr.mean(1).reshape((4, 1))

array([[-1.0979837 ,  1.14900695, -0.05102325],
       [-0.04538757, -0.09975984,  0.14514741],
       [-0.52911337,  0.03146244,  0.49765093],
       [-0.32184845, -0.65679726,  0.97864571]])

Using `reshape` can be tedious, so Numpy offers a more convenient syntax



In [10]:
arr = np.zeros((4, 4, 2))
arr3 = arr[:, :, None]
arr3.shape

(4, 4, 1, 2)

Suppose we had a 3-D array and wanted to subtract the mean along the z-axis



In [11]:
arr = np.random.randn(3, 4, 5)
arrm = arr.mean(2)
arrm.shape

(3, 4)

In [12]:
arrd = arr - arrm[:, :, np.newaxis]
arrd.mean(2)

array([[-5.55111512e-17,  0.00000000e+00, -2.22044605e-17,
         0.00000000e+00],
       [-1.66533454e-17,  1.11022302e-17,  2.22044605e-17,
         0.00000000e+00],
       [-4.44089210e-17, -2.22044605e-17, -3.33066907e-17,
         0.00000000e+00]])

## Vectorizing functions



-   We saw some functions that operate on the entire array (e.g. `np.sqrt`)
    -   These element-wise transformation functions are called *unary* or **ufuncs**
    -   How do we convert a regular Python scalar function to a "Numpy" function (i.e., `ufuncs`)?
    -   Think `math.exp` versus `np.exp`



In [13]:
def addsubtract(a, b):
    """Add two variables if first is greater, subtract otherwise."""
    if a > b:
        return a + b
    else:
        return a - b

In [14]:
addsubtract(3, 2)

5

In [15]:
addsubtract([0,3,6,9], [1,3,5,7])

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [16]:
vec_addsubtract = np.vectorize(addsubtract)

In [17]:
vec_addsubtract([0,3,6,9], [1,3,5,7])

array([-1,  0, 11, 16])

### But what about optimization?



[numba](https://numba.pydata.org/numba-doc/dev/index.html) to the rescue!



In [18]:
def naive_sum(x):
    s = 0.0
    for i in range(len(x)):
        s += x[i]
    return s

In [19]:
from numba import jit

@jit(nopython=True)
def fast_sum(x):
    s = 0.0
    for i in range(len(x)):
        s += x[i]
    return s

In [20]:
arr = np.random.randn(100000000)
%time naive_sum(arr)

CPU times: user 5.44 s, sys: 256 µs, total: 5.44 s
Wall time: 5.44 s


-1644.7970576716891

In [21]:
%time fast_sum(arr)

CPU times: user 169 ms, sys: 4.01 ms, total: 173 ms
Wall time: 188 ms


-1644.7970576716891

## ☛ Hands-on exercises



Let's write a 1-D linear interpolation function! For an unknown function $y = f(x)$ we want to linearly interpolate for $x_n$ with $$y_n = f(x_n) = f(x_0) + \dfrac{f(x_1) - f(x_0)}{x_1 - x_0} (x_n - x_0)$$ where $(x_0, x_1)$ is an interval that contains $x_n$



In [22]:
x = np.sort(np.random.randn(100))
y = x**2 - 2 * x

First write a function that doesn't use any `numpy` functions apart from `np.zeros`.



In [23]:
def interp1(x, y, x1):
    """Interpolate function y when calculated at x, at x1.
    
    x: array of x values
    y: function evaluations at x
    x1: array of values where y will be interpolated at
    """
    y1 = np.zeros(len(x1))
    for k, xnew in enumerate(x1):
        for i in range(len(x)):
            if xnew > x[i]:
                j = i
        if j >= len(x)-1:
            y1[k] = np.nan
        else:
            y1[k] = (y[j+1] - y[j]) / (x[j+1] - x[j]) * (xnew - x[j]) + y[j]
    return y1

Next write a function that uses `numpy` function (**not `interp`!!!**)



In [24]:
def interp(x, y, x1):
    y1 = np.zeros(len(x1))
    for k, xnew in enumerate(x1):
        try:
            j = np.where(xnew > x)[0][-1]
            y1[k] = (y[j+1] - y[j]) / (x[j+1] - x[j]) * (xnew - x[j]) + y[j]
        except:
            y1[k] = np.nan
    return y1

@jit(nopython=True)
def ginterp(x, y, x1):
    y1 = np.zeros(len(x1))
    for k, xnew in enumerate(x1):
        try:
            j = np.where(xnew > x)[0][-1]
            y1[k] = (y[j+1] - y[j]) / (x[j+1] - x[j]) * (xnew - x[j]) + y[j]
        except:
            y1[k] = np.nan
    return y1

Time the two functions above and then use the `jit` decorator on your first function and time them again



In [25]:
x1 = np.random.randn(10000000)
#%time out = interp1(x, y, x1)
%time out = interp(x, y, x1)
%time out = ginterp(x, y, x1)

CPU times: user 14.7 s, sys: 10 ms, total: 14.7 s
Wall time: 14.7 s
CPU times: user 2.22 s, sys: 17 ms, total: 2.24 s
Wall time: 2.25 s


Now use the `np.interp` function and time it



In [28]:
%time out = np.interp(x1, x, y)

CPU times: user 268 ms, sys: 5 ms, total: 273 ms
Wall time: 272 ms


Form the following array



In [29]:
np.array([[0, 1, 2], [10, 11, 12], [20, 21, 22], [30, 31, 32]])

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

by adding the following arrays



In [30]:
a = np.array([0, 10, 20, 30])
print(a)
b = np.array([0, 1, 2])
print(b)

[ 0 10 20 30]
[0 1 2]


In [31]:
a[:, np.newaxis] + b[np.newaxis, :]

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

## Scientific Python



> SciPy is a collection of mathematical algorithms and convenience functions built on the NumPy extension of Python#+attr<sub>ipynb</sub>: (slideshow . ((slide<sub>type</sub> . subslide)))

| Subpackage|Description|
|---|---|
| cluster|Clustering algorithms|
| constants|Physical constants|
| fftpack|Fast Fourier Transforms|
| integrate|Integration and ODEs|
| interpolate|Interpolation|
| io|Input/Output|
| linalg|Linear algebra|
| ndimage|Image processing|
| optimize|Optimization|
| signal|Signal processing|
| sparse|Sparse matrices|
| spatial|Spatial data structures and algorithms|
| special|Special function|
| stats|Statistical distributions and functions|



## Scipy example: Optimization



We will minimize the Rosenbrock function $$f(x) = \sum_{i=2}^{N}100 (x_{i+1} -
 x_{i}^{2})^{2} - (x1-x_{i})^{2}$$ which has a minimum of 0 at $x_{i}=1$



In [33]:
from scipy.optimize import minimize

In [34]:
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [35]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [36]:
res.x

array([1., 1., 1., 1., 1.])